In [1]:
!pip install /kaggle/input/monai-110/monai-1.2.0-202306081546-py3-none-any.whl
try:
    import monai
    print(monai.__version__)
    print("Install Success")
except:
    print("Install Monai Error")

Processing /kaggle/input/monai-110/monai-1.2.0-202306081546-py3-none-any.whl


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

1.2.0
Install Success


In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import torch
import nibabel as nib

In [3]:
df_train = pd.read_csv('/kaggle/input/rsna-2023-abdominal-trauma-detection/train.csv')
df_train

,patient_id,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,liver_high,spleen_healthy,spleen_low,spleen_high,any_injury
0,10004,1,0,0,1,0,1,0,1,0,0,0,0,1,1
1,10005,1,0,1,0,1,0,0,1,0,0,1,0,0,0
2,10007,1,0,1,0,1,0,0,1,0,0,1,0,0,0
3,10026,1,0,1,0,1,0,0,1,0,0,1,0,0,0
4,10051,1,0,1,0,1,0,0,1,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3142,9951,1,0,1,0,1,0,0,1,0,0,1,0,0,0
3143,9960,1,0,1,0,1,0,0,1,0,0,1,0,0,0
3144,9961,1,0,1,0,1,0,0,1,0,0,1,0,0,0
3145,9980,1,0,1,0,1,0,0,1,0,0,0,0,1,1


In [4]:
df_train_meta = pd.read_csv('/kaggle/input/rsna-2023-abdominal-trauma-detection/train_series_meta.csv')
df_train_meta

,patient_id,series_id,aortic_hu,incomplete_organ
0,10004,21057,146.00,0
1,10004,51033,454.75,0
2,10005,18667,187.00,0
3,10007,47578,329.00,0
4,10026,29700,327.00,0
...,...,...,...,...
4706,9961,2003,381.00,0
4707,9961,63032,143.75,0
4708,9980,40214,103.00,0
4709,9980,40466,135.00,0


In [5]:
patient_ids = np.unique(df_train['patient_id'])

In [6]:
segmentation_root = Path('/kaggle/input/rsna-2023-abdominal-trauma-detection/segmentations')

In [7]:
seged_series_ids = [folder.stem for folder in segmentation_root.iterdir()]

In [8]:
def find_patient_row_by_series_id(df, series_id):
    patient_id = df.loc[df['series_id'] == series_id]
    return patient_id

def find_patient_id_by_series_id(df, series_id):
    patient_id = df.loc[df['series_id'] == series_id]
    return int(patient_id['patient_id'])


In [9]:
seged_patient_ids = [find_patient_row_by_series_id(df_train_meta, int(series)) for series in seged_series_ids]

In [10]:
seged_patient_ids[:5]

[      patient_id  series_id  aortic_hu  incomplete_organ
 2976       47065      39222     366.75                 0,
       patient_id  series_id  aortic_hu  incomplete_organ
 2596       42008      52961      209.0                 0,
      patient_id  series_id  aortic_hu  incomplete_organ
 782       19410       7334       96.0                 0,
       patient_id  series_id  aortic_hu  incomplete_organ
 3839       57444      15415      322.0                 0,
      patient_id  series_id  aortic_hu  incomplete_organ
 306       13794      51141      190.0                 0]

In [11]:
class seg(torch.utils.data.Dataset):
    def __init__(self, seged_patient_ids, seg_path, image_path):
        if not isinstance(seg_path, Path):
            seg_path = Path(seg_path)
        if not isinstance(image_path, Path):
            image_path = Path(image_path)
        self.seged_patient_ids = seged_patient_ids
        self.seg_path = seg_path
        self.image_path = image_path
        
    def __len__(self,):
        return len(seged_patient_ids)
    
    def __getitem__(self, idx):
        item = self.seged_patient_ids[idx]
        image = self.image_path / str(int(item['patient_id'])) / str(int(item['series_id']))
        seg = self.seg_path / str(int(item['patient_id'])) / str(int(item['series_id']))
        return image, seg

In [12]:
train_ds = seg(seged_patient_ids,'/kaggle/input/rsna-2023-abdominal-trauma-detection/segmentations', '/kaggle/input/rsna-2023-abdominal-trauma-detection/train_images')

In [13]:
# import pydicom
# from pathlib import Path
# import matplotlib.pyplot as plt
# import imageio

# # 指定DICOM文件所在文件夹
# dicom_dir = Path('/kaggle/input/rsna-2023-abdominal-trauma-detection/train_images/10004/21057')

# # 获取文件夹下所有.dcm文件，并按照文件名排序
# dicom_files = sorted(dicom_dir.glob('*.dcm'))

# # 读取DICOM文件并转换为像素数组
# images = [pydicom.dcmread(str(dicom_file)).pixel_array for dicom_file in dicom_files]

# # 调整像素数组的对比度
# images = [img.astype(float) / img.max() * 255 for img in images]

# # 创建gif文件
# imageio.mimsave('/kaggle/working/animation.gif', images, 'GIF', duration=0.1)

# # 在jupyter notebook中展示gif

In [14]:
# 读取nii文件
nii_file = nib.load('/kaggle/input/rsna-2023-abdominal-trauma-detection/segmentations/10000.nii')

# 转换为像素数组
nii_data = nii_file.get_fdata()

# 获取像素数组的形状
height, width, depth = nii_data.shape
print(height, width, depth)

512 512 483


In [15]:
find_patient_row_by_series_id(df_train_meta, 51033)

,patient_id,series_id,aortic_hu,incomplete_organ
1,10004,51033,454.75,0


In [16]:
import pydicom
import nibabel as nib
from pathlib import Path
import matplotlib.pyplot as plt
import imageio
import numpy as np

# 读取NIfTI文件
nii_file = nib.load('/kaggle/input/rsna-2023-abdominal-trauma-detection/segmentations/51033.nii')
nii_data = nii_file.get_fdata()
# nii_data = np.flip(nii_data, axis=1)  # 翻转第一个轴
nii_data = np.flip(nii_data, axis=2)  # 翻转第一个轴
# nii_data = np.transpose(nii_data, (1, 0, 2))  # 交换第一个和第二个轴

# 指定DICOM文件所在文件夹
dicom_dir = Path('/kaggle/input/rsna-2023-abdominal-trauma-detection/train_images') / str(find_patient_id_by_series_id(df_train_meta, 51033)) / '51033'

# 获取文件夹下所有.dcm文件，并按照文件名排序
dicom_files = sorted(dicom_dir.glob('*.dcm'))
print(len(dicom_files))
print(len(sorted(dicom_dir.glob('*'))))
# 读取DICOM文件并转换为像素数组
dicom_images = [pydicom.dcmread(str(dicom_file)).pixel_array for dicom_file in dicom_files]

# 确保NIfTI文件和DICOM文件有相同数量的切片
assert len(dicom_images) == nii_data.shape[2], "Mismatch in number of slices between DICOM and NIfTI files."
# print(np.unique(nii_data))
# 创建一个存储每一帧的列表
frames = []

# 对每一层进行处理
for i in range(len(dicom_images)):

    # 创建一个新的图像
    fig, ax = plt.subplots(1,1)
    
    # 绘制DICOM切片
    ax.imshow(dicom_images[i], cmap=plt.cm.bone)
    
    # 在DICOM切片之上绘制NIfTI切片
    ax.imshow(nii_data[:, :, i].T, alpha=0.2)
    
    # 隐藏坐标轴
    ax.axis('off')

    # 将图像保存到内存中
    fig.canvas.draw()
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    
    # 关闭图像，释放内存
    plt.close(fig)
    
    # 将图像添加到帧列表中
    frames.append(image)

# 创建gif文件
imageio.mimsave('/kaggle/working/animation_calib3.gif', frames, 'GIF', duration=0.1)


1044
1044


In [17]:
dicom_dir

PosixPath('/kaggle/input/rsna-2023-abdominal-trauma-detection/train_images/10004/51033')